In [103]:
import os
import re
import csv
import h5py
import warnings
import tqdm
import numpy as np
import pandas as pd

In [102]:
dataFolder = "D:/Rafael/Dados"

allUnitsFolder = os.listdir(dataFolder+"/Dat")

allUnits = [re.findall("A.", unit) for unit in allUnitsFolder] # Get all folder names with "Unidade"
allUnits = set([name[0][-1] for name in allUnits if len(name)>0]) # Filter for unique models

with h5py.File(f"{dataFolder}/dataset.hdf5", "w") as fModel:
    for unitNum in tqdm.tqdm(allUnits,desc = " Compressor", position=0):
        
        unitGrp = fModel.create_group(unitNum) # Create new group for each compressor unit

        r = re.compile(f"A{unitNum}.*")
        unitFiles = list(filter(r.match,allUnitsFolder))


        for testFile in tqdm.tqdm(unitFiles, desc = "  Teste", leave=False,  position=1):
            testName = os.path.splitext(testFile)[0]

            testGrp = unitGrp.create_group(testName)

            testTags = testName.replace("[","").replace("]","").split("-")
            testGrp.attrs['type'] = testTags[0][0]
            testGrp.attrs['RPM'] = testTags[1]
            testGrp.attrs['rep'] = testTags[2]
            testGrp.attrs['Tsuc'] = testTags[4]
            testGrp.attrs['Tdis'] = testTags[5]



            


 Compressor:   0%|          | 0/5 [00:00<?, ?it/s]

 Compressor: 100%|██████████| 5/5 [00:01<00:00,  2.58it/s]
